In [22]:
def load_model(path, model):
    # load pretrained model and update weights
    pretrained_dict = torch.load(path)
    model_dict = model.state_dict()
    # 1. filter out unnecessary keys
    pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
    # 2. overwrite entries in the existing state dict
    model_dict.update(pretrained_dict)
    # 3. load the new state dict
    model.load_state_dict(model_dict)
    return model

In [2]:
def adam(params, config=None):
    if config is None:
        config = {
            'lr': 3e-5,
            'warmup_proportion': 0.1,
            'weight_decay': 0.01
        }
        
    no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']

    optimizer_grouped_parameters = [
        {'params': [p for n, p in params if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
        {'params': [p for n, p in params if any(nd in n for nd in no_decay)], 'weight_decay': 0}
    ]

    optim = Bert.optimization.BertAdam(optimizer_grouped_parameters,
                                       lr=config['lr'],
                                       warmup=config['warmup_proportion'])
    return optim

In [3]:
import sys
sys.path.insert(1, '../')
from sklearn.model_selection import train_test_split
from utils.dataset import EHRDatasetCodePrediction
from model.tokenizer import EHRTokenizer
import pytorch_pretrained_bert as Bert
from torch.utils.data import DataLoader
from model.model import *
from utils.config import BertConfig
from model.model2 import BertMultiLabelPrediction
from model.trainer import PatientTrajectoryPredictor
import pytorch_lightning as pl
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [4]:
path = '../processing/readmission_data_ccsr'

In [5]:
data = pd.read_parquet(path)

In [6]:
data['visit_len'] = data['hadm_id'].apply(lambda x: len(x))
datamorethan1visit = data[data['visit_len'] > 1]
datamorethan1visit['ccsr_traincodes'] = datamorethan1visit['ccsr'].apply(lambda x: x[:-1])
datamorethan1visit['ndc_labels'] = datamorethan1visit['ndc'].apply(lambda x: x[-1])

In [7]:
'''
labelar = []

def labelvoc(labels):
    for a in labels:
        labelar.append(a)
'''

'\nlabelar = []\n\ndef labelvoc(labels):\n    for a in labels:\n        labelar.append(a)\n'

In [8]:
#datamorethan1visit['ndc_labels'].apply(lambda x: labelvoc(x))

In [9]:
#np.save('../processing/labelsvoc_ndc.npy', np.array(labelar))

In [10]:
global_params = {
    'max_seq_len': 32,
    'gradient_accumulation_steps': 1
}

optim_param = {
    'lr': 3e-5,
    'warmup_proportion': 0.1,
    'weight_decay': 0.01
}

train_params = {
    'batch_size': 64,
    'use_cuda': True,
    'max_len_seq': global_params['max_seq_len'],
    'device': 'cuda' #change this to run on cuda #'cuda:0'
}

In [11]:
tokenizer = EHRTokenizer(task='ndc')

In [12]:
model_config = {
    'vocab_size': len(tokenizer.getVoc('code').keys()), # number of disease + symbols for word embedding
    'hidden_size': 300, # word embedding and seg embedding hidden size
    'seg_vocab_size': 2, # number of vocab for seg embedding
    'age_vocab_size': len(tokenizer.getVoc('age').keys()), # number of vocab for age embedding,
    'gender_vocab_size': 3,
    'max_position_embeddings': train_params['max_len_seq'], # maximum number of tokens
    'hidden_dropout_prob': 0.1, # dropout rate
    'num_hidden_layers': 2, # number of multi-head attention layers required
    'num_attention_heads': 4, # number of attention heads
    'attention_probs_dropout_prob': 0.1, # multi-head attention dropout rate
    'intermediate_size': 300, # the size of the "intermediate" layer in the transformer encoder
    'hidden_act': 'gelu', # The non-linear activation function in the encoder and the pooler "gelu", 'relu', 'swish' are supported
    'initializer_range': 0.02, # parameter weight initializer range
}

In [13]:
class BertConfig(Bert.modeling.BertConfig):
    def __init__(self, config):
        super(BertConfig, self).__init__(
            vocab_size_or_config_json_file=config.get('vocab_size'),
            hidden_size=config['hidden_size'],
            num_hidden_layers=config.get('num_hidden_layers'),
            num_attention_heads=config.get('num_attention_heads'),
            intermediate_size=config.get('intermediate_size'),
            hidden_act=config.get('hidden_act'),
            hidden_dropout_prob=config.get('hidden_dropout_prob'),
            attention_probs_dropout_prob=config.get('attention_probs_dropout_prob'),
            max_position_embeddings = config.get('max_position_embeddings'),
            initializer_range=config.get('initializer_range'),
        )
        self.seg_vocab_size = config.get('seg_vocab_size')
        self.age_vocab_size = config.get('age_vocab_size')
        self.gender_vocab_size = config.get('gender_vocab_size')

In [14]:
conf = BertConfig(model_config)
tensorboarddir = '../logs/'

In [15]:
trainset, testset = train_test_split(datamorethan1visit, test_size=0.2)

In [16]:
labelvoc = tokenizer.getVoc('label')

In [17]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer(classes=list(labelvoc.values()))
mlb.fit([[each] for each in list(labelvoc.values())])

MultiLabelBinarizer(classes=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14,
                             15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27,
                             28, 29, ...])

In [18]:
traindata = EHRDatasetCodePrediction(trainset, max_len=train_params['max_len_seq'], tokenizer=tokenizer, prediction_task='ndc') 
testdata = EHRDatasetCodePrediction(testset, max_len=train_params['max_len_seq'], tokenizer=tokenizer, prediction_task='ndc') 

In [19]:
trainer = pl.Trainer(
        max_epochs=5, 
        gpus=-1,
        logger=pl.loggers.TensorBoardLogger(save_dir=tensorboarddir),
        callbacks=[pl.callbacks.progress.TQDMProgressBar()], 
        progress_bar_refresh_rate=1,
        weights_summary=None, # Can be None, top or full
        num_sanity_val_steps=10,
    )

trainloader = torch.utils.data.DataLoader(traindata, batch_size=train_params['batch_size'], shuffle=True, num_workers=3)
testloader = torch.utils.data.DataLoader(testdata, batch_size=train_params['batch_size'], shuffle=True, num_workers=3)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [20]:
model = BertMultiLabelPrediction(conf, num_labels=len(labelvoc.keys())) 
PATH = "../saved_models/Shallow_unsuffled"
model = load_model(PATH, model)
params = list(model.named_parameters())
optim = adam(params, optim_param)

t_total value of -1 results in schedule not being applied


In [21]:
patienttrajectory = PatientTrajectoryPredictor(model, optim, optim_param, train_objective='visit', metrics=True, binarizer=mlb)

trainer.fit(
    patienttrajectory, 
    train_dataloader=trainloader,
);

predictions = trainer.predict(patienttrajectory, dataloaders=testloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 0:   1%|▍                                                 | 18/1825 [00:19<31:53,  1.06s/it, loss=0.65, v_num=166]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 18it [00:00, ?it/s]

In [22]:
aucpr = 0
for pred in predictions:
    aucpr+=pred['AUCPR']
aucpr / len(predictions)

0.09521388212687404

In [23]:
auc = 0
for pred in predictions:
    auc+=pred['AUC']
auc / len(predictions)

0.5